In [2]:
import os
import pandas as pd

# 폰트설정
import matplotlib.font_manager as fm
from matplotlib import rc

rc('font', family=fm.FontProperties(fname='C:/Users/HYUK/AppData/Local/Microsoft/Windows/Fonts/NotoSansKR-Thin.ttf').get_name())


dir_path = "../predict_csv/"

# 해당 위치에 존재하는 파일들을 리스트로 반환하는 함수
def get_file_list(directory):
    file_list = []

    # 디렉터리 내의 모든 항목에 대해 반복합니다.
    for item in os.listdir(directory):
        # 전체 경로를 생성합니다.
        item_path = os.path.join(directory, item)

        # 파일인 경우에만 리스트에 추가합니다.
        if os.path.isfile(item_path):
            file_list.append(item)

    return file_list

file_path = get_file_list(dir_path)
print(file_path)

def read_csv(dir_path : str, file_path : str, do_print : bool = True):
    df = pd.read_csv(dir_path+file_path)
    if do_print :
        print(file_path,"\n")
        print("데이터 프레임 구조:", df.shape,"\n")
        print(df.head())
        print("\n데이터 정보 :\n")
        print(df.info())
        print("\nNan 값 존재 데이터")
        print(df[df.isnull().any(axis=1)])
    return df

def save_csv(dir_path : str, file_path : str,  df : pd.DataFrame ):
    new_data_path = dir_path+file_path
    
    data_path_os = os.path.dirname(new_data_path)
    if not os.path.exists(data_path_os):
        os.makedirs(data_path_os)

    
    # 데이터 저장
    df.to_csv(dir_path+file_path, index=False)

['Predict.csv']


In [3]:
df = read_csv(dir_path,"Predict.csv")

Predict.csv 

데이터 프레임 구조: (232, 7) 

   미국금리  실업률  기업대출  가계대출  주택매매가격지수_변동율  통화량_변동율  기준금리
0  1.00  3.7   1.9   1.8          -0.8      0.6  3.75
1  2.25  3.8   1.8   1.7          -0.3      0.6  3.25
2  3.00  3.7   2.4   1.9           0.5      0.3  3.25
3  3.25  3.8   1.8   1.5           0.8      1.1  3.25
4  3.25  3.8   2.0   1.6           0.7      0.9  3.25

데이터 정보 :

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   미국금리          232 non-null    float64
 1   실업률           232 non-null    float64
 2   기업대출          232 non-null    float64
 3   가계대출          232 non-null    float64
 4   주택매매가격지수_변동율  232 non-null    float64
 5   통화량_변동율       232 non-null    float64
 6   기준금리          232 non-null    float64
dtypes: float64(7)
memory usage: 12.8 KB
None

Nan 값 존재 데이터
Empty DataFrame
Columns: [미국금리, 실업률, 기업대출, 가계대출, 주택매매가격지수_변동율, 통화량_변동율, 기준금리]